**INSTALL PACKAGES**

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml

In [4]:
!pip install html5lib

In [5]:
!pip install requests

**IMPORT BEAUTIFULSOUP, REQUESTS, PANDAS, AND NUMPY**

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [7]:
#get wiki link
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#use beautiful soup to scrape the wiki page
soup = BeautifulSoup(source, 'html5lib')

In [8]:
#create a table with the scraped data
table1 = soup.find('table',{'class':'wikitable sortable'})
data1 = table1.find_all('td')

In [9]:
#append the wiki page data to the table
counter = 0
table = []
row = []
for details in data1:
    if counter < 3:
        row.append(details.get_text().strip())
        counter+=1
    else:
        table.append(row)
        counter = 0
        row = []
        row.append(details.get_text().strip())
        counter+=1
#define headers 
headers=['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(table, columns=headers)

#clean the data 
def cleaning (row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row
df1 = df.apply(cleaning, axis=1)
df1 = df1[df1['Borough'] != 'Not assigned']

In [10]:
#combine neighbourhoods with same postal code in one row
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df2.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
df2.shape

(103, 3)

In [19]:
!pip install geocoder

In [20]:
import geocoder # import geocoder

def get_coordinates1 (row):
    # initialize your variable to None
    lat_lng_coords = None
    print(row['Postcode'])
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
        lat_lng_coords = g.latlng
        print('1')

    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
    print(row['Latitude'], row['Longitude'])
    return row

In [21]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f389ca35935b4f8e982b864138e3fe53 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9qd8J6nujvR2eD3BYsZsF0MVx8A3hChJWtqNAPuKw0mv',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_f389ca35935b4f8e982b864138e3fe53.get_object(Bucket='capstoneweek1-donotdelete-pr-q6fsoogtqc1ljz',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

coordinates_df = pd.read_csv(body)
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
def get_coordinates2 (row):
    temperature_df = coordinates_df[coordinates_df['Postal Code'] == row['Postcode']]
    row['Latitude'] = float(temperature_df['Latitude'])
    row['Longitude'] = float(temperature_df['Longitude'])
    return row
    
df_with_coordinates = df2
df_with_coordinates['Latitude'] = np.nan
df_with_coordinates['Longitude'] = np.nan
df_with_coordinates = df_with_coordinates.apply(get_coordinates2, axis=1)
df_with_coordinates.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [29]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [32]:
import folium
# create map of New York using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
df_toronto_neighbourhoods = df_with_coordinates
df_toronto_neighbourhoods['Neighbourhood'] = '['+ df_toronto_neighbourhoods['Postcode'] + '] ' + df_toronto_neighbourhoods['Neighbourhood']
# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto_neighbourhoods['Latitude'], df_toronto_neighbourhoods['Longitude'], df_toronto_neighbourhoods['Borough'], df_toronto_neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough.upper())
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [33]:
CLIENT_ID = '2SWA5CPXPLPAI5KLPCATASAXPBHFGEOBEJ0AKTCBJ1QBHMUD' #'client-ID' Foursquare ID
CLIENT_SECRET = 'SAZCKB2C0K02XE0Y3WTORUAUTFZ5ELWLUUZWPFIOVRUZAPSU' #  Foursquare Secret
VERSION = '20181231'
print('My Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My Credentails:
CLIENT_ID: 2SWA5CPXPLPAI5KLPCATASAXPBHFGEOBEJ0AKTCBJ1QBHMUD
CLIENT_SECRET:SAZCKB2C0K02XE0Y3WTORUAUTFZ5ELWLUUZWPFIOVRUZAPSU


In [34]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
df_toronto_venues = getNearbyVenues(names=df_toronto_neighbourhoods['Neighbourhood'],
                                   latitudes=df_toronto_neighbourhoods['Latitude'],
                                   longitudes=df_toronto_neighbourhoods['Longitude'])
print(df_toronto_venues.shape)
df_toronto_venues.groupby('Neighbourhood').count()

[M1B] Rouge, Malvern
[M1C] Highland Creek, Rouge Hill, Port Union
[M1E] Guildwood, Morningside, West Hill
[M1G] Woburn
[M1H] Cedarbrae
[M1J] Scarborough Village
[M1K] East Birchmount Park, Ionview, Kennedy Park
[M1L] Clairlea, Golden Mile, Oakridge
[M1M] Cliffcrest, Cliffside, Scarborough Village West
[M1N] Birch Cliff, Cliffside West
[M1P] Dorset Park, Scarborough Town Centre, Wexford Heights
[M1R] Maryvale, Wexford
[M1S] Agincourt
[M1T] Clarks Corners, Sullivan, Tam O'Shanter
[M1V] Agincourt North, L'Amoreaux East, Milliken, Steeles East
[M1W] L'Amoreaux West
[M1X] Upper Rouge
[M2H] Hillcrest Village
[M2J] Fairview, Henry Farm, Oriole
[M2K] Bayview Village
[M2L] Silver Hills, York Mills
[M2M] Newtonbrook, Willowdale
[M2N] Willowdale South
[M2P] York Mills West
[M2R] Willowdale West
[M3A] Parkwoods
[M3B] Don Mills North
[M3C] Flemingdon Park, Don Mills South
[M3H] Bathurst Manor, Downsview North, Wilson Heights
[M3J] Northwood Park, York University
[M3K] CFB Toronto, Downsview East
[M

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"[M1B] Rouge, Malvern",1,1,1,1,1,1
"[M1C] Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"[M1E] Guildwood, Morningside, West Hill",7,7,7,7,7,7
[M1G] Woburn,4,4,4,4,4,4
[M1H] Cedarbrae,7,7,7,7,7,7
[M1J] Scarborough Village,1,1,1,1,1,1
"[M1K] East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4
"[M1L] Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"[M1M] Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2


In [36]:
print('There are {} uniques categories.'.format(len(df_toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = df_toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_onehot.shape

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"[M1B] Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"[M1C] Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"[M1C] Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"[M1E] Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"[M1E] Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"[M1B] Rouge, Malvern",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[M1C] Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[M1E] Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,[M1G] Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,[M1H] Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Top 5 Venues based on Postcode**

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----[M1B] Rouge, Malvern----
                      venue  freq
0      Fast Food Restaurant   1.0
1         Accessories Store   0.0
2               Music Venue   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----[M1C] Highland Creek, Rouge Hill, Port Union----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2           Accessories Store   0.0
3           Mobile Phone Shop   0.0
4               Movie Theater   0.0


----[M1E] Guildwood, Morningside, West Hill----
                 venue  freq
0          Pizza Place  0.14
1   Mexican Restaurant  0.14
2         Intersection  0.14
3    Electronics Store  0.14
4  Rental Car Location  0.14


----[M1G] Woburn----
                             venue  freq
0                      Coffee Shop  0.50
1                Korean Restaurant  0.25
2                Indian Restaurant  0.25
3                Accessories Store  0.00
4  Molecular Gastronomy Restaurant  0.00


-

**Common Venues based on Postcode**

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[M1B] Rouge, Malvern",Fast Food Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,"[M1C] Highland Creek, Rouge Hill, Port Union",Construction & Landscaping,Bar,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,"[M1E] Guildwood, Morningside, West Hill",Pizza Place,Medical Center,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Empanada Restaurant,Ethiopian Restaurant,Eastern European Restaurant
3,[M1G] Woburn,Coffee Shop,Indian Restaurant,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,[M1H] Cedarbrae,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


**CLUSTER**

In [48]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [49]:
# add clustering labels
try:
    neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    pass

toronto_merged = df_toronto_neighbourhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels']#.astype(int)
toronto_merged['Cluster Labels'].dropna(inplace=True)
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"[M1B] Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M1C,Scarborough,"[M1C] Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
2,M1E,Scarborough,"[M1E] Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Medical Center,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Empanada Restaurant,Ethiopian Restaurant,Eastern European Restaurant
3,M1G,Scarborough,[M1G] Woburn,43.770992,-79.216917,0.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
4,M1H,Scarborough,[M1H] Cedarbrae,43.773136,-79.239476,0.0,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Bakery,Hakka Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [50]:
toronto_merged['Cluster Labels'].value_counts()

0.0    83
5.0    11
1.0     2
2.0     1
4.0     1
3.0     1
Name: Cluster Labels, dtype: int64

In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters